In [60]:
# import dependencies 
import numpy as np 
import pandas as pd 
import string
from string import digits
from nltk.corpus import stopwords
import nltk
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
#Download the necessary datasets
#nltk.download()

In [16]:
# read in data 
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [17]:
# describe the datatset 
df.describe()

<ipython-input-17-9068c8991ab7>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


,category,headline,authors,link,short_description,date
count,200853,200853,200853,200853,200853,200853
unique,41,199344,27993,200812,178353,2309
top,POLITICS,Sunday Roundup,,https://www.huffingtonpost.comhttps://www.wash...,,2013-01-17 00:00:00
freq,32739,90,36620,2,19712,100
first,NaN,NaN,NaN,NaN,NaN,2012-01-28 00:00:00
last,NaN,NaN,NaN,NaN,NaN,2018-05-26 00:00:00


In [58]:
# number by catergories 
df['category'].value_counts() #.plot(kind='barh')

POLITICS          32739
WELLNESS          17827
ENTERTAINMENT     16058
TRAVEL             9887
STYLE & BEAUTY     9649
PARENTING          8677
HEALTHY LIVING     6694
QUEER VOICES       6314
FOOD & DRINK       6226
BUSINESS           5937
COMEDY             5175
SPORTS             4884
BLACK VOICES       4528
HOME & LIVING      4195
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3651
WOMEN              3490
IMPACT             3459
DIVORCE            3426
CRIME              3405
MEDIA              2815
WEIRD NEWS         2670
GREEN              2622
WORLDPOST          2579
RELIGION           2556
STYLE              2254
SCIENCE            2178
WORLD NEWS         2177
TASTE              2096
TECH               2082
MONEY              1707
ARTS               1509
FIFTY              1401
GOOD NEWS          1398
ARTS & CULTURE     1339
ENVIRONMENT        1323
COLLEGE            1144
LATINO VOICES      1129
CULTURE & ARTS     1030
EDUCATION          1004
Name: category, 

In [39]:
# text pre-processing
# There are actually many methods to convert a corpus to a vector format. 
# The simplest is the the bag-of-words approach, where each unique word in a text will be represented by one number.

# remove punctuation & lower 
df['headline_cleansed'] = df['headline'].apply(lambda x: (''.join([char for char in x if char not in string.punctuation])).lower())

# remove numeric characters 
df['headline_cleansed'] = df['headline_cleansed'].str.replace('\d+', '')

# stem words 
porter = PorterStemmer()
df['headline_cleansed'] = df['headline_cleansed'].apply(lambda x: porter.stem(x))

# remove any stopwords
df['headline_cleansed'] = df['headline_cleansed'].apply(lambda x: [word for word in x.split() if word not in stopwords.words('english')] )

# create the sentence with spaces 
df['headline_cleansed'] = df['headline_cleansed'].apply(lambda x: ' '.join(x))

# sample output 
df['headline_cleansed'].head()

0                    mass shootings texas last week tv
1    smith joins diplo nicky jam world cups officia...
2                    hugh grant marries first time age
3    jim carrey blasts castrato adam schiff democra...
4    julianna margulies uses donald trump poop bags...
Name: headline_cleansed, dtype: object

In [65]:
# Now we can use .transform on our Bag-of-Words (bow) transformed object and transform the entire DataFrame of messages. 
# So what is TF-IDF?
# TF-IDF stands for term frequency-inverse document frequenc
# TF: Term Frequency, which measures how frequently a term occurs in a document. 
# Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones.
# TF: Term Frequency, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization:
# TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
# IDF: Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following:
# IDF(t) = log_e(Total number of documents / Number of documents with term t in it).

msg_train, msg_test, label_train, label_test = \
train_test_split(df['headline_cleansed'], df['category'], test_size=0.2, random_state=42)

In [66]:
pipeline = Pipeline([
    ('bow', CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [67]:
pipeline.fit(msg_train,label_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [68]:
predictions = pipeline.predict(msg_test)

In [69]:
print (classification_report(predictions,label_test))

C:\Users\Lewis James\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Lewis James\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

          ARTS       0.00      0.00      0.00         0
ARTS & CULTURE       0.00      0.00      0.00         0
  BLACK VOICES       0.00      0.67      0.01         6
      BUSINESS       0.05      0.79      0.10        76
       COLLEGE       0.00      0.00      0.00         0
        COMEDY       0.04      0.91      0.08        47
         CRIME       0.06      0.78      0.11        49
CULTURE & ARTS       0.01      1.00      0.02         2
       DIVORCE       0.14      0.95      0.24       100
     EDUCATION       0.00      0.00      0.00         0
 ENTERTAINMENT       0.76      0.43      0.55      5616
   ENVIRONMENT       0.04      1.00      0.08        12
         FIFTY       0.00      0.00      0.00         0
  FOOD & DRINK       0.50      0.71      0.59       887
     GOOD NEWS       0.00      0.00      0.00         0
         GREEN       0.00      1.00      0.01         2
HEALTHY LIVING       0.00      0.38      0.00  

C:\Users\Lewis James\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [70]:
svm_pipeline = Pipeline([
    ('bow', CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('svm_classifier', SGDClassifier()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [71]:
svm_pipeline.fit(msg_train,label_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('svm_classifier', SGDClassifier())])

In [72]:
svm_predictions = svm_pipeline.predict(msg_test)

In [73]:
print (classification_report(svm_predictions,label_test))

                precision    recall  f1-score   support

          ARTS       0.11      0.23      0.15       150
ARTS & CULTURE       0.11      0.29      0.15       105
  BLACK VOICES       0.27      0.48      0.35       503
      BUSINESS       0.35      0.45      0.40       864
       COLLEGE       0.30      0.46      0.36       158
        COMEDY       0.33      0.54      0.41       656
         CRIME       0.48      0.48      0.48       655
CULTURE & ARTS       0.18      0.43      0.25        93
       DIVORCE       0.68      0.70      0.69       676
     EDUCATION       0.19      0.40      0.26        91
 ENTERTAINMENT       0.69      0.55      0.61      3982
   ENVIRONMENT       0.19      0.37      0.25       140
         FIFTY       0.07      0.21      0.11        95
  FOOD & DRINK       0.69      0.56      0.62      1562
     GOOD NEWS       0.12      0.26      0.17       133
         GREEN       0.20      0.42      0.27       245
HEALTHY LIVING       0.08      0.27      0.12  

In [83]:
# test model on BBC headlines 
svm_pipeline.predict(np.array(["four men arrested in anti Semitism video investigation"]))

array(['CRIME'], dtype='<U14')

In [81]:
svm_pipeline.predict(np.array(["restaurants and pubs are reopening inside but what are the rules"]))

array(['TRAVEL'], dtype='<U14')

In [82]:
svm_pipeline.predict(np.array(["the mid-air walkways saving endangered animals"]))

array(['GREEN'], dtype='<U14')